<a href="https://colab.research.google.com/github/R0N3ldrt/Thesis/blob/main/spectrum-1_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading Enviroment

## Loading Drive and folders

In [1]:
# BLOQUE PARA USAR DESDE COLAB

# Google drive loading as work station for local-usage of the files.
from google.colab import drive
drive.mount('/content/gdrive',force_remount= True)

#-----------------------------------------------------------------------------

Mounted at /content/gdrive


In [2]:
# Lista para cambiar los paths rapido.
workers = ["Ronald", "Local"]

# Cambiar el numero aqui para cambiar los paths.
worker = workers[0]

if worker == "Ronald":
  path= "/content/gdrive/MyDrive/Thesis_Workstation/ANN_dataset"
else: path = os.getcwd()

## Loading Necesary Libraries

In [3]:
# Importing necesary libraries
import os, time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import pickle
from sklearn.linear_model import LinearRegression

import warnings
warnings.simplefilter("ignore", UserWarning)
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Loading Data and Pre-Procesing

In [4]:
data_path=path+"/Spectrum"

qpsk_path="/QPSK/A.10hops100km/"
qam_path="/16QAM/A.10hops100km"

#file_name="/dataSet_Spectrum_Samples_QPSK_37GHz_LongHaul_output_10x100.xlsx"
file_name="/QPSK_dummy.xlsx"
# Skipping first row since only contain "Frequency" headers
df_qpsk=pd.read_excel(data_path+qpsk_path+file_name, sheet_name = "Sheet1", skiprows=1)

#file_name="/dataSet_Spectrum_Samples_16QAM_75GHz_LongHaul_output_10x100.xlsx"
file_name="/16QAM_dummy.xlsx"
df_qam=pd.read_excel(data_path+qam_path+file_name, sheet_name = "Sheet1", skiprows=1)

#Diff in column 5 to 1,286 0.03125

In [5]:
#dump_file = dict desireed to be saved
#filename = just the name of the file (file saved on "pickle_folder")

def pickle_out(dump_file, filename):
  filepath = path+"/pickle_folder/"+str(filename)+".pickle"
  pickleout = open(filepath, "wb")
  pickle.dump(dump_file, pickleout)
  pickleout.close()

def pickle_in(filename):
  filepath = path+"/pickle_folder/"+str(filename)+".pickle"
  picklein = open(filepath, "rb")
  loaded_dict = pickle.load(picklein)
  return loaded_dict

## Aggregator

In [6]:
def prime_number_finder(stop_gap, total_num_of_data):
  prime_nums = []
  i=2
  while i <= stop_gap:
    if (total_num_of_data % i==0):
      prime_nums.append(i)
    i+=1
  return prime_nums

def aggreagator_selection(stop_gap=25, total_num_of_data=1281):
  current_freq_offset = 0.03125
  
  # Find the prime numbers where the total_num_of_data is divisible
  prime_nums = prime_number_finder(stop_gap, total_num_of_data)
  
  # Find the offsets of the new aggregators
  new_freq_offsets = []
  for nums in prime_nums:
    freq_offset = current_freq_offset*(nums-1)
    new_freq_offsets.append(freq_offset)

  # Ask the user for the desired aggregator
  print('Please select one of the following prime number to conduct the aggregation {} with frequencies offsets of {}(PHz) respectively. (Current Freq offset is 0.03125 PHz)'.format(prime_nums, new_freq_offsets))
  while True:
    user_selection = input('Prime number to conduct the aggregation? ')
    if int(user_selection) not in prime_nums:
      print('Please select one of the available prime numbers. (Current selection is not avaliable)')
    else:
      break
  return int(user_selection)

def aggreagator(df, stop_gap=25, total_num_of_data=1281):
  agg_num = aggreagator_selection(stop_gap, total_num_of_data)
  
  # Creating new df of aggregate values
  agg_df = pd.DataFrame()
  agg_df = df[df.columns[0:5]]

  agg=0
  loop_cnt=0
  init_column_cnt = 5
  while agg<=total_num_of_data:
    loop_cnt+=1
    # Obtain current last columns stop
    agg=agg_num+init_column_cnt
    # Select working columns
    new_df = df[df.columns[init_column_cnt:agg]]

    init_column_cnt += (agg_num)
    #print(init_column_cnt)
    # Get average of freq values for new header
    headers = list(new_df.columns.values)
    headers_array = np.asarray(headers)
    new_header_name = np.mean(headers_array)

    # Add aggragated values to new df
    agg_df[str(new_header_name)] = new_df.mean(axis = 1)
  agg_dict = agg_df.to_dict()
  return agg_df, agg_dict


In [7]:
# QPSK AGG
stop = 7
QPSK_agg_df, QPSK_agg_dict = aggreagator(df_qpsk, stop_gap=100, total_num_of_data=1281)
pickle_out(QPSK_agg_dict, "test_agg_spectrum_QPSK_"+str(stop))

Please select one of the following prime number to conduct the aggregation [3, 7, 21, 61] with frequencies offsets of [0.0625, 0.1875, 0.625, 1.875](PHz) respectively. (Current Freq offset is 0.03125 PHz)
Prime number to conduct the aggregation? 3


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:57: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


In [8]:
# QPSK QAM
stop = 7
QAM_agg_df, QAM_agg_dict = aggreagator(df_qam, stop_gap=100, total_num_of_data=1281)
pickle_out(QAM_agg_dict, "agg_spectrum_QAM_"+str(stop))

Please select one of the following prime number to conduct the aggregation [3, 7, 21, 61] with frequencies offsets of [0.0625, 0.1875, 0.625, 1.875](PHz) respectively. (Current Freq offset is 0.03125 PHz)
Prime number to conduct the aggregation? 3


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:57: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


In [9]:
pickle_dummy_dict = pickle_in("agg_spectrum_QPSK_7")

print(pickle_dummy_dict.keys())
pickle_dummy_dict['-19.90625'].values()

dict_keys(['sampleid', 'Channels', 'Distance_km', 'power_dBm', '#node', '-19.90625', '-19.6875', '-19.46875', '-19.25', '-19.03125', '-18.8125', '-18.59375', '-18.375', '-18.15625', '-17.9375', '-17.71875', '-17.5', '-17.28125', '-17.0625', '-16.84375', '-16.625', '-16.40625', '-16.1875', '-15.96875', '-15.75', '-15.53125', '-15.3125', '-15.09375', '-14.875', '-14.65625', '-14.4375', '-14.21875', '-14.0', '-13.78125', '-13.5625', '-13.34375', '-13.125', '-12.90625', '-12.6875', '-12.46875', '-12.25', '-12.03125', '-11.8125', '-11.59375', '-11.375', '-11.15625', '-10.9375', '-10.71875', '-10.5', '-10.28125', '-10.0625', '-9.84375', '-9.625', '-9.40625', '-9.1875', '-8.96875', '-8.75', '-8.53125', '-8.3125', '-8.09375', '-7.875', '-7.65625', '-7.4375', '-7.21875', '-7.0', '-6.78125', '-6.5625', '-6.34375', '-6.125', '-5.90625', '-5.6875', '-5.46875', '-5.25', '-5.03125', '-4.8125', '-4.59375', '-4.375', '-4.15625', '-3.9375', '-3.71875', '-3.5', '-3.28125', '-3.0625', '-2.84375', '-2.625

dict_values([-52.58722602859268, -51.777602184478255, -53.00329057493259, -55.85847203872787, -54.2264411002062, -54.25347624625919, -53.365185015537925, -55.05460739835035, -52.92877612025617, -52.734421550389584, -58.74401826893385, -48.63746686499979, -56.44660670882369, -48.27247617292802, -53.83972599255794, -54.86937274897692, -55.57411278690153, -50.50396703594872, -49.60572093699432, -48.25319561224723, -52.59046914587442, -57.682477991353736, -52.252890223122314, -50.8665340356822, -54.364867952225744, -53.78109273171549, -56.467180752635876, -54.85119620354811, -51.14296407836939, -52.152817983536906, -53.782106603484635, -53.73058069785552, -58.21894447645663, -52.03413220002404, -57.14900409612067, -51.53686851443755, -56.55314159430914, -55.86559874058809, -50.81783926823658, -53.23959353985477, -55.558113381428825, -54.21257727879089, -52.67069707227461, -52.89647545667034, -56.07426908330729, -54.99347767231384, -54.90215012347203, -50.34181293124205, -53.14101521054793,

##Dummy Model

In [124]:
def get_dummy_spectrum(df):
  new_df = df.groupby(['#node']).mean()
  new_df.drop(['sampleid', 'Channels', 'power_dBm'], axis = 1, inplace = True)
  dummy_dict = new_df.to_dict()
  
  return new_df, dummy_dict

In [125]:
#new_df = 
#node	| Distance_km	|-20	      |-19.96875	|-19.9375																						
#1	    | 0.0	        |-52.955056	|-54.126681	|-52.713840	
#2	    | 100.0	      |-51.813386	|-53.194155	|-52.697528
#3	    | 200.0	      |-55.626777	|-54.634329	|-54.702958

#dummy_dict.keys() = dict_keys(['Distance_km', -20, -19.96875, -19.9375, -19.90625, -19.875, -19.84375...
#dummy_dict[-20].values() = dict_values([-52.955056039, -51.813386274, -55.626777088])

In [126]:
# Dummy Model QPSK
QPSK_dummy_df, QPSK_dummy_dict = get_dummy_spectrum(QPSK_agg_df)
pickle_out(QPSK_dummy_dict, "dummy_AGG_spectrum_QPSK")

In [127]:
# Dummy Model QAM
QAM_dummy_df, QAM_dummy_dict = get_dummy_spectrum(QAM_agg_df)
pickle_out(QAM_dummy_dict, "dummy_AGG_spectrum_QAM")

In [128]:
pickle_dummy_dict = pickle_in("dummy_AGG_spectrum_QPSK")
pickle_dummy_dict['Distance_km'].values()

dict_values([0.0, 100.0, 200.0])

# Processing

In [15]:
# Aggregated model
pickle_agg_QPSK_dict = pickle_in("test_agg_spectrum_QPSK_7")
pickle_agg_AQM_dict = pickle_in("test_agg_spectrum_QAM_7")

# Dummy models
pickle_dummy_QPSK_dict = pickle_in("test_dummy_AGG_spectrum_QPSK")
pickle_dummy_QAM_dict = pickle_in("test_dummy_AGG_spectrum_QAM")

In [16]:
#new_df = 
#node	  | Distance_km	|-20	      |-19.96875	|-19.9375																						
#1	    | 0.0	        |-52.955056	|-54.126681	|-52.713840	
#2	    | 100.0	      |-51.813386	|-53.194155	|-52.697528
#3	    | 200.0	      |-55.626777	|-54.634329	|-54.702958

#dummy_dict.keys() = dict_keys(['Distance_km', -20, -19.96875, -19.9375, -19.90625, -19.875, -19.84375...
#dummy_dict[-20].values() = dict_values([-52.955056039, -51.813386274, -55.626777088])

In [17]:
print(pickle_agg_QPSK_dict.keys())
pickle_dummy_QAM_dict['-39.90625'].values()

dict_keys(['sampleid', 'Channels', 'Distance_km', 'power_dBm', '#node', '-19.96875', '-19.875', '-19.78125', '-19.6875', '-19.59375', '-19.5', '-19.40625', '-19.3125', '-19.21875', '-19.125', '-19.03125', '-18.9375', '-18.84375', '-18.75', '-18.65625', '-18.5625', '-18.46875', '-18.375', '-18.28125', '-18.1875', '-18.09375', '-18.0', '-17.90625', '-17.8125', '-17.71875', '-17.625', '-17.53125', '-17.4375', '-17.34375', '-17.25', '-17.15625', '-17.0625', '-16.96875', '-16.875', '-16.78125', '-16.6875', '-16.59375', '-16.5', '-16.40625', '-16.3125', '-16.21875', '-16.125', '-16.03125', '-15.9375', '-15.84375', '-15.75', '-15.65625', '-15.5625', '-15.46875', '-15.375', '-15.28125', '-15.1875', '-15.09375', '-15.0', '-14.90625', '-14.8125', '-14.71875', '-14.625', '-14.53125', '-14.4375', '-14.34375', '-14.25', '-14.15625', '-14.0625', '-13.96875', '-13.875', '-13.78125', '-13.6875', '-13.59375', '-13.5', '-13.40625', '-13.3125', '-13.21875', '-13.125', '-13.03125', '-12.9375', '-12.84375'

dict_values([-51.967543919, -50.006399713])

In [18]:
def freq_selector(pickle_dict):
  invalid = {'Distance_km','sampleid', 'Channels', 'power_dBm', '#node'}
  freqs = []
  for i in pickle_dict:
    if i not in invalid:
      freqs.append(i)
  return np.array(freqs)
#freq_selector(pickle_agg_QPSK_dict)

In [184]:
def focus_selector_calc(eval_df):
  
  sensitivity_df = pd.DataFrame()

  # Find min common multipler (smallesr we can break the slots)
  non_escencial_data_col = 5
  
  number_of_data = eval_df.shape[1]-(non_escencial_data_col-1)-1
  
  # Selecting the prime factors where we can reduce the data
  #min_prime_num = min(prime_number_finder(stop_gap=100, total_num_of_data=number_of_data))
  user_prime_selection = aggreagator_selection(stop_gap=100, total_num_of_data=number_of_data)  

  cnt = 1
  current = non_escencial_data_col
  while current <= number_of_data:
    end_range = current+(user_prime_selection)
    new_df = eval_df[eval_df.columns[current:end_range]]

    
    # Add aggragated values to new df
    sensitivity_df[eval_df.columns[current]] = new_df.mean(axis = 1)
    current+=user_prime_selection
    cnt+=1
  return sensitivity_df, user_prime_selection

# Select only the columns where we observe and increase of 500%
# Return a dictionary with the row as keys and and array with positional location of columns to be selected as values
def focus_selector_range(df, threshhold=5):
  old_column = 0
  selected_data_ranges = {}
  shape = df.shape
  rows = shape[0]
  columns = shape[1]
  for row in range(rows):
    delta = 0
    col_selected_array = []  
    for col in range(columns):
      if col == 0:
        pass
      else:   
        delta = abs((df.iat[row, col])-(df.iat[row, (col-1)]))   
        if delta>=threshhold:
          col_selected_array.append(col) 
    selected_data_ranges[row] = col_selected_array
   
  return selected_data_ranges

# Return dictionary with rows and columns to be selected from sensitivity_df
def focus_selector_removing_outlayers(df, focus_dict, node):
  threshold_pass = True
  new_dict = {}
  unique_columns = set({})
  info_per_node = 10
  nodes_dict_of_selected_columns = {}

  # k = row
  # v_array = array of columns selected
  for k, v_array in focus_dict.items():
    new_array = []
    old_v = 0

    # Check if selected columns are at least 7 or more to make sure threshold is not losing data
    if len(v_array) <= 7:
      #print('Threshold on function "focus_selector_range" is to big please select and smaller threshold.') 
      threshold_pass = False
      break
    # Adding one more selection earlier for being more flexible
    elif old_v == 0:
      min_array = min(v_array)-1
      new_array.append(min_array)
    
    for v in v_array:
      if abs(v - old_v) <=5 and abs(v - old_v) > 1 and old_v!= 0:
        x = abs(v - old_v)
        for i in range(x-1):
          new_array.append(old_v+(i+1))
      # Expanding by one the sensitive area
      elif abs(v - old_v)>25:
        new_array.append(old_v+1)
        new_array.append(v-1)

      old_v = v
      new_array.append(v)

    # Adding one more selection in the end for being more flexible
    max_array = max(new_array)+1
    new_array.append(max_array)

    # Adding storage of all columns saved for later function
    unique_columns.update(new_array)
    if k == (info_per_node-1):
      nodes_dict_of_selected_columns[node] = unique_columns
      info_per_node = node*info_per_node
      unique_columns = set({})
    new_dict[k] = new_array
  

  return new_dict, nodes_dict_of_selected_columns, threshold_pass

#-17.28125	-16.625	-15.96875
#{0: [3, 4, 5, 6, 7, 54, 55, 56, 57]

In [171]:
def focus_selector(sensitivity_df, original_df, nodes_dict_of_selected_columns, node_selected):

  total_of_columns = (sensitivity_df.shape[1])
  mid_point = total_of_columns/2

  # Get list of columns to be selected
  selected_node_columns = list(nodes_dict_of_selected_columns[node_selected])
  
  #Cleaning unwanted columns
  max_pos = selected_node_columns.index(max(selected_node_columns))
  
  #Final selected columns for selected node
  columns_to_be_selected = selected_node_columns[:max_pos+1]

  # Select only the desired columns 
  columns_to_be_selected = sorted(columns_to_be_selected)

  # Separate data in to left and right curves
  left_columns = list()
  rigth_columns = list()

  for col in columns_to_be_selected:
    if col <= mid_point:
      left_columns.append(col)
    else:
      if col != sensitivity_df.shape[1]:
        rigth_columns.append(col)

  df_left = sensitivity_df.iloc[:, left_columns]
  df_rigth = sensitivity_df.iloc[:, rigth_columns]
  
  # Add back additional info
  selection = original_df.loc[:,'sampleid']
  df_left.insert(0, 'sampleid', selection)
  df_rigth.insert(0, 'sampleid', selection)

  selection = original_df.loc[:,'Channels']
  df_left.insert(1, 'Channels', selection)
  df_rigth.insert(1, 'Channels', selection)

  selection = original_df.loc[:,'Distance_km']
  df_left.insert(2, 'Distance_km', selection)
  df_rigth.insert(2, 'Distance_km', selection)

  selection = original_df.loc[:,'power_dBm']
  df_left.insert(3, 'power_dBm', selection)
  df_rigth.insert(3, 'power_dBm', selection)

  selection = original_df.loc[:,'#node']
  df_left.insert(4, '#node', selection)
  df_rigth.insert(4, '#node', selection)

  #Select only the node desired
  node_df_left = df_left.loc[df_left['#node'] == int(node_selected)]
  node_df_rigth = df_rigth.loc[df_rigth['#node'] == int(node_selected)]

  return node_df_left, node_df_rigth

In [118]:
'''
sensitivity_df, agg_range = focus_selector_calc(df_qpsk)

# Using threshold of 800%
focus_dict = focus_selector_range(sensitivity_df, threshhold=8)
new_dict_focus_selector, nodes_dict_of_selected_columns, threshold_pass = focus_selector_removing_outlayers(sensitivity_df, focus_dict)

if threshold_pass == True:
  print('Selected columns for each row:')
  print(len(nodes_dict_of_selected_columns[1]))
  print(nodes_dict_of_selected_columns[1])
  node_df_left, node_df_rigth = focus_selector(sensitivity_df, df_qpsk, new_dict_focus_selector, node_selected = 1)
'''

"\nsensitivity_df, agg_range = focus_selector_calc(df_qpsk)\n\n# Using threshold of 800%\nfocus_dict = focus_selector_range(sensitivity_df, threshhold=8)\nnew_dict_focus_selector, nodes_dict_of_selected_columns, threshold_pass = focus_selector_removing_outlayers(sensitivity_df, focus_dict)\n\nif threshold_pass == True:\n  print('Selected columns for each row:')\n  print(len(nodes_dict_of_selected_columns[1]))\n  print(nodes_dict_of_selected_columns[1])\n  node_df_left, node_df_rigth = focus_selector(sensitivity_df, df_qpsk, new_dict_focus_selector, node_selected = 1)\n"

In [192]:
def focus_selector_complete(original_df, node_to_be_eval, min_desired_reduction_of_noise=0.15, aggregation = True):

  original_col_num = (original_df.shape[1]-5)
  # Decide if we want agregation
  if aggregation == True:
    sensitivity_df, agg_range = focus_selector_calc(original_df)
    max_smothed_amt = int(original_col_num/agg_range)
  else:
    sensitivity_df = original_df.copy()
    drop_cols = [0, 1, 2, 3, 4]
    sensitivity_df.drop(sensitivity_df.columns[drop_cols], axis=1, inplace=True)

    max_smothed_amt = original_col_num

  old_val_threshold = 0
  for threshhold_val in range(20):

    focus_dict = focus_selector_range(sensitivity_df, threshhold=threshhold_val)
    new_dict_focus_selector, nodes_dict_of_selected_columns, threshold_pass = focus_selector_removing_outlayers(sensitivity_df, focus_dict, node = node_to_be_eval)

    if threshold_pass == True:
      percentage_reduced_of_noise = 1-(len(nodes_dict_of_selected_columns[node_to_be_eval])/max_smothed_amt)

      # Find the optimal threshold
      if percentage_reduced_of_noise >= min_desired_reduction_of_noise:
        print('Reduce {} columns to {} columns.'.format(original_col_num, len(nodes_dict_of_selected_columns[node_to_be_eval])))
        print('{}% reduced noise using a threshold of {}%.'.format(round(percentage_reduced_of_noise,2)*100, (threshhold_val*100)))
      
        print('Columns selected {}'.format(nodes_dict_of_selected_columns[node_to_be_eval]))
        node_df_left, node_df_rigth = focus_selector(sensitivity_df, original_df, nodes_dict_of_selected_columns, node_selected = node_to_be_eval)
        break  

      # Stop in case we arraive to the maximum threshold that would be looped
      elif threshhold_val == 20:
        print('Arrived to maximum threshold to be checked. (2,000%)')
        print('Reduce {} columns to {} columns.'.format(original_col_num, len(nodes_dict_of_selected_columns[node_to_be_eval])))
        print('{}% reduced noise using a threshold of {}%.'.format(round(percentage_reduced_of_noise,2)*100, (threshhold_val*100)))
      
        print('Columns selected {}'.format(nodes_dict_of_selected_columns[node_to_be_eval]))
        node_df_left, node_df_rigth = focus_selector(sensitivity_df, original_df, nodes_dict_of_selected_columns, node_selected = node_to_be_eval)
        break          
    
    # In case the threholds is to big we stop and use the previos one
    elif threshold_pass == False:
      focus_dict = focus_selector_range(sensitivity_df, threshhold=old_val_threshold)
      new_dict_focus_selector, nodes_dict_of_selected_columns, threshold_pass = focus_selector_removing_outlayers(sensitivity_df, focus_dict) 
           
      percentage_reduced_of_noise = 1-(len(nodes_dict_of_selected_columns[node_to_be_eval])/max_smothed_amt)

      print('Started to lose significant data keeping previos threshold value.')
      print('Reduce {} columns to {} columns.'.format(original_col_num, len(nodes_dict_of_selected_columns[node_to_be_eval])))
      print('{}% reduced noise using a threshold of {}%.'.format(round(percentage_reduced_of_noise,2)*100, (old_val_threshold*100)))
    
      print('Columns selected {}'.format(nodes_dict_of_selected_columns[node_to_be_eval]))
      node_df_left, node_df_rigth = focus_selector(sensitivity_df, original_df, nodes_dict_of_selected_columns, node_selected = node_to_be_eval)
      break  


    old_val_threshold += 1
  return node_df_left, node_df_rigth, nodes_dict_of_selected_columns


In [193]:
node_df_left, node_df_rigth, nodes_dict_of_selected_columns = focus_selector_complete(df_qpsk, node_to_be_eval=2, min_desired_reduction_of_noise=0.15, aggregation = True)

Please select one of the following prime number to conduct the aggregation [3, 7, 21, 61] with frequencies offsets of [0.0625, 0.1875, 0.625, 1.875](PHz) respectively. (Current Freq offset is 0.03125 PHz)
Prime number to conduct the aggregation? 3


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


Reduce 1281 columns to 352 columns.
18.0% reduced noise using a threshold of 800%.
Columns selected {1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 23, 24, 25, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 56, 57, 59, 60, 61, 63, 64, 65, 66, 67, 69, 70, 71, 72, 73, 74, 75, 76, 77, 81, 82, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232

In [203]:
def model_focus_selector(model_df, nodes_dict_of_selected_columns, node_to_be_eval):

  node_selected_for_sample = list(nodes_dict_of_selected_columns.keys())[0]
  
  # Pre-process dummy model
  model_node_row = model_df.iloc[[node_to_be_eval-1]]
  model_node_row.drop('Distance_km', axis=1, inplace=True)
  
  # Select columns that are saved in the sample data
  selected_columns = list(nodes_dict_of_selected_columns[node_selected_for_sample])
  model_node_row.iloc[:, selected_columns]
  
  # Separating model in left and right curves
  total_of_columns = (model_node_row.shape[1])
  mid_point = total_of_columns/2

  left_columns = list()
  rigth_columns = list()
  col_array = nodes_dict_of_selected_columns[node_selected_for_sample]

  for col in selected_columns:
    if col <= mid_point and col in col_array:
      left_columns.append(col)
    else:
      if col != model_node_row.shape[1] and col in col_array:
        rigth_columns.append(col)
  #print(left_columns)
  model_node_row_left = model_node_row.iloc[:, left_columns]
  model_node_row_right = model_node_row.iloc[:, rigth_columns]
  
  # Add back additional info
  model_node_row_left.insert(0, 'sampleid', 1)
  model_node_row_right.insert(0, 'sampleid', 1)

  model_node_row_left.insert(1, 'Channels', 1)
  model_node_row_right.insert(1, 'Channels', 1)

  model_node_row_left.insert(2, 'Distance_km', node_to_be_eval*100)
  model_node_row_right.insert(2, 'Distance_km', node_to_be_eval*100)

  model_node_row_left.insert(3, 'power_dBm', -2)
  model_node_row_right.insert(3, 'power_dBm', -2)

  model_node_row_left.insert(4, '#node', node_to_be_eval)
  model_node_row_right.insert(4, '#node', node_to_be_eval)


  return model_node_row_left, model_node_row_right

In [198]:
node_df_left, node_df_rigth, nodes_dict_of_selected_columns = focus_selector_complete(df_qpsk, node_to_be_eval=2, min_desired_reduction_of_noise=0.15, aggregation = True)

Please select one of the following prime number to conduct the aggregation [3, 7, 21, 61] with frequencies offsets of [0.0625, 0.1875, 0.625, 1.875](PHz) respectively. (Current Freq offset is 0.03125 PHz)
Prime number to conduct the aggregation? 3


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


Reduce 1281 columns to 352 columns.
18.0% reduced noise using a threshold of 800%.
Columns selected {1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 23, 24, 25, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 56, 57, 59, 60, 61, 63, 64, 65, 66, 67, 69, 70, 71, 72, 73, 74, 75, 76, 77, 81, 82, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232

In [201]:
model_node_row_left, model_node_row_right = model_focus_selector(QPSK_dummy_df, nodes_dict_of_selected_columns, node_to_be_eval=1)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


## Display of new df

In [204]:
display(model_node_row_left)

,sampleid,Channels,Distance_km,power_dBm,#node,-19.875,-19.78125,-19.6875,-19.59375,-19.5,...,-1.125,-1.03125,-0.9375,-0.84375,-0.75,-0.375,-0.28125,-0.1875,-0.09375,0.0
#node,,,,,,,,,,,,,,,,,,,,,
1,1,1,100,-2,1,-53.652772,-54.412841,-52.411059,-52.705496,-53.970664,...,-12.026442,-12.0063,-11.609852,-11.210192,-11.482319,-11.975944,-11.247961,-9.792938,-13.079584,-11.57299


In [205]:
display(node_df_left)

,sampleid,Channels,Distance_km,power_dBm,#node,-19.90625,-19.8125,-19.71875,-19.625,-19.53125,...,-1.15625,-1.0625,-0.96875,-0.875,-0.78125,-0.40625,-0.3125,-0.21875,-0.125,-0.03125
10,11,1,100,-2,2,-60.717547,-56.398255,-53.466966,-55.489984,-52.813161,...,-13.124516,-13.064955,-19.878310,-14.092949,-14.774573,-12.456316,-12.400204,-15.380878,-9.478390,-12.518852
11,12,1,100,-2,2,-45.154631,-54.036290,-51.033799,-51.752648,-49.302803,...,-14.630213,-9.678197,-10.306654,-11.037745,-13.509072,-7.737781,-14.452396,-11.822110,-10.192461,-11.846299
12,13,1,100,-2,2,-53.271146,-57.677338,-54.015429,-55.466907,-51.861905,...,-19.801212,-16.257726,-6.087445,-16.220888,-12.211688,-15.967935,-11.735137,-9.915330,-13.068275,-12.777089
13,14,1,100,-2,2,-46.900471,-53.688198,-60.588875,-51.906944,-49.999406,...,-7.310856,-12.777552,-20.225135,-8.817696,-16.346859,-14.087443,-9.289306,-7.255323,-15.590994,-15.007235
14,15,1,100,-2,2,-55.247101,-53.745921,-56.200103,-53.819230,-54.238498,...,-8.790394,-14.447070,-7.491303,-14.676820,-10.513413,-18.223181,-7.372109,-19.152016,-18.517830,-9.618819
15,16,1,100,-2,2,-51.912873,-57.347325,-51.278512,-57.367606,-49.575955,...,-10.807059,-9.013090,-16.061713,-14.092356,-5.251080,-12.147487,-9.426831,-5.037879,-12.949783,-12.489177
16,17,1,100,-2,2,-53.910756,-53.671796,-58.875833,-54.313467,-58.279569,...,-9.782579,-12.635099,-9.402970,-12.374379,-12.847590,-13.018388,-12.807221,-10.576889,-12.106825,-10.323982
17,18,1,100,-2,2,-51.319447,-53.997868,-51.182574,-58.193663,-54.240875,...,-13.253770,-11.502975,-9.785302,-7.492864,-10.441769,-10.827107,-19.467531,-6.097846,-15.939057,-10.718276
18,19,1,100,-2,2,-51.100547,-56.063445,-52.671966,-52.007881,-53.720627,...,-13.006206,-13.295873,-9.427220,-7.518905,-10.461342,-5.024252,-7.464636,-7.977249,-14.112024,-13.498649
19,20,1,100,-2,2,-47.781522,-48.983736,-57.731805,-46.974644,-57.885939,...,-10.319302,-7.572447,-7.642800,-6.911382,-8.596809,-9.896669,-8.576413,-4.524880,-8.680326,-7.185841


In [94]:
display(model_node_row_right)

,sampleid,Channels,Distance_km,power_dBm,#node,0.09375,0.1875,0.28125,0.375,0.46875,...,18.5625,19.03125,19.125,19.21875,19.3125,19.40625,19.5,19.59375,19.6875,19.78125
#node,,,,,,,,,,,,,,,,,,,,,
1,1,1,100,-2,1,-12.317919,-10.911665,-11.193323,-10.005633,-11.467842,...,-52.24085,-55.574644,-53.45745,-54.487223,-54.74286,-56.035196,-51.657352,-53.887728,-54.595005,-54.693334


In [89]:
display(df_qpsk)

,sampleid,Channels,Distance_km,power_dBm,#node,-20,-19.96875,-19.9375,-19.90625,-19.875,...,19.71875,19.75,19.78125,19.8125,19.84375,19.875,19.90625,19.9375,19.96875,20
0,1,1,0,-2,1,-50.396272,-51.195507,-52.325501,-57.685476,-51.730270,...,-50.701297,-50.425586,-57.242411,-54.978886,-58.984450,-52.289897,-50.007652,-56.562352,-50.661037,-57.857937
1,2,1,0,-2,1,-49.464198,-62.300068,-49.878752,-45.576386,-50.935177,...,-52.919979,-61.894104,-60.851791,-53.621297,-47.427855,-57.669031,-51.735427,-56.145979,-51.259631,-46.965564
2,3,1,0,-2,1,-52.100342,-54.825356,-49.933739,-53.418630,-55.604328,...,-52.437768,-48.572056,-58.180955,-50.003337,-49.935404,-52.854423,-52.839831,-56.938673,-48.511327,-53.268586
3,4,1,0,-2,1,-61.429210,-54.973444,-45.026184,-63.731247,-45.706566,...,-56.792207,-59.099399,-45.632244,-56.454966,-46.126678,-50.632960,-48.675022,-50.899182,-64.439508,-46.552580
4,5,1,0,-2,1,-51.777530,-57.344578,-55.337847,-52.396041,-53.875442,...,-48.008932,-51.073828,-49.675923,-57.762095,-49.534186,-59.334809,-59.237640,-62.395677,-46.607586,-69.244091
5,6,1,0,-2,1,-56.348628,-48.323410,-52.791372,-48.463897,-58.102067,...,-63.216051,-50.084285,-56.313096,-61.528461,-52.478272,-50.024323,-45.508370,-47.952339,-50.220176,-47.461991
6,7,1,0,-2,1,-59.530214,-46.793059,-54.027619,-52.906629,-53.718027,...,-54.985099,-58.129401,-60.080767,-53.301332,-51.950843,-59.423618,-57.730467,-65.814835,-54.484546,-57.792294
7,8,1,0,-2,1,-52.818323,-57.433254,-50.022994,-59.626194,-58.581050,...,-63.592384,-47.710070,-50.966537,-55.692830,-47.676732,-52.455189,-68.033712,-48.267217,-51.216616,-51.337803
8,9,1,0,-2,1,-46.037314,-57.909915,-63.732986,-52.182822,-50.834665,...,-57.325800,-56.238710,-54.520079,-46.396466,-54.367238,-49.434797,-49.168688,-48.523573,-49.754015,-49.321008
9,10,1,0,-2,1,-49.648529,-50.168222,-54.061412,-59.438983,-57.902770,...,-59.078244,-59.259655,-66.459379,-48.650078,-50.762140,-47.733430,-49.128781,-54.307126,-57.668304,-56.184202


In [91]:
display(node_df_rigth)

,sampleid,Channels,Distance_km,power_dBm,#node,0.0625,0.15625,0.25,0.34375,0.4375,...,18.53125,19.0,19.09375,19.1875,19.28125,19.375,19.46875,19.5625,19.65625,19.75
0,1,1,0,-2,1,-8.560761,-14.192841,-15.308988,-6.232919,-15.718079,...,-53.549533,-53.391188,-57.003342,-54.180432,-55.929226,-53.685770,-54.573121,-54.901276,-53.117014,-54.215628
1,2,1,0,-2,1,-10.591460,-6.819910,-10.564098,-8.830513,-15.817850,...,-51.537441,-52.675644,-55.382418,-53.290998,-48.686969,-53.426423,-48.667324,-51.275908,-54.835198,-58.789064
2,3,1,0,-2,1,-8.283382,-10.283430,-13.450209,-12.778612,-9.993028,...,-48.611839,-53.606160,-56.652709,-56.285192,-51.693924,-64.989293,-49.395410,-50.925655,-55.505864,-52.252116
3,4,1,0,-2,1,-13.669836,-11.168663,-5.218217,-11.375746,-7.522850,...,-52.968103,-53.151827,-53.870560,-54.640180,-59.415261,-54.676540,-53.838356,-48.210381,-54.552774,-53.728870
4,5,1,0,-2,1,-18.587664,-10.522540,-9.703829,-8.370643,-11.134089,...,-51.777777,-57.344211,-57.100780,-58.379422,-55.335203,-56.729537,-52.077778,-53.626389,-53.366190,-52.837282
5,6,1,0,-2,1,-16.029550,-10.092535,-11.146011,-4.177009,-12.758659,...,-51.522328,-59.745445,-47.420478,-56.587717,-57.485052,-52.485296,-50.829762,-51.890006,-58.026839,-55.975281
6,7,1,0,-2,1,-15.256261,-10.549735,-13.169865,-11.278147,-6.919431,...,-55.519578,-55.139514,-48.759233,-58.287394,-53.312507,-55.226176,-52.680751,-63.977246,-53.399970,-57.170500
7,8,1,0,-2,1,-4.180230,-10.145045,-9.834341,-10.959290,-14.235732,...,-53.433582,-58.191199,-56.279264,-51.803747,-55.710205,-54.214841,-53.931745,-57.111652,-56.118792,-51.456479
8,9,1,0,-2,1,-23.035372,-13.552319,-9.884707,-13.781820,-12.206824,...,-50.798371,-55.432531,-54.322663,-50.403793,-57.905826,-62.237995,-49.307877,-48.097190,-49.944873,-52.385085
9,10,1,0,-2,1,-4.984669,-11.789633,-13.652961,-12.271633,-8.371881,...,-52.689950,-57.068722,-47.783049,-51.013356,-51.954424,-52.680089,-51.271392,-58.861577,-57.082538,-58.123037


In [92]:
display(QPSK_dummy_df)

,Distance_km,-19.96875,-19.875,-19.78125,-19.6875,-19.59375,-19.5,-19.40625,-19.3125,-19.21875,...,19.03125,19.125,19.21875,19.3125,19.40625,19.5,19.59375,19.6875,19.78125,19.875
#node,,,,,,,,,,,,,,,,,,,,,
1,0.0,-53.265193,-53.652772,-54.412841,-52.411059,-52.705496,-53.970664,-53.049249,-53.813891,-54.089213,...,-55.574644,-53.457450,-54.487223,-54.742860,-56.035196,-51.657352,-53.887728,-54.595005,-54.693334,-52.438729
2,100.0,-52.568357,-51.731604,-54.561017,-54.704586,-53.729298,-53.191874,-52.556893,-53.251489,-53.559093,...,-52.079893,-54.024187,-53.376551,-53.561515,-53.628489,-53.416521,-53.282656,-54.797403,-53.550991,-53.279212
3,200.0,-54.988021,-52.898298,-54.227910,-53.253689,-52.783244,-52.414599,-51.787709,-51.678843,-52.072996,...,-54.215778,-53.147360,-53.885240,-52.387626,-52.193880,-53.277343,-53.192047,-53.694075,-53.411451,-52.683480


## Linear Regresion

In [206]:
#node_df_left
#node_df_rigth
#model_node_row_left
#model_node_row_right

#print(node_df_left.shape[0])
#print(node_df_left.shape[1])

def linear_regresion(node_df_left, node_df_rigth, model_node_row_left, model_node_row_right):

  # Get model data
  model_node_row_left = model_node_row_left.iloc[0].to_numpy()
  model_node_row_left = model_node_row_left[5:]
  y = np.array(model_node_row_left)

  # Get Sample data
  for row in range(node_df_left.shape[0]):
    row_data_sample = node_df_left.iloc[row].to_numpy()
    row_data_sample = row_data_sample[5:]
    x = np.array(row_data_sample).reshape((-1, 1))
    
    # Get linear regresion
    print('---------------')
    print('Sample {}'.format(row+1))  
    model = LinearRegression()
    model.fit(x, y)
    model = LinearRegression().fit(x, y)
    r_sq = model.score(x, y)
    print('coefficient of determination:', r_sq)
    print('intercept:', model.intercept_)
    print('slope:', model.coef_)
    print('---------------')
  
  
  # Prediction
  #y_pred = model.predict(x)
  #print('predicted response:', y_pred, sep='\n')


In [207]:
linear_regresion(node_df_left, node_df_rigth, model_node_row_left, model_node_row_right)

---------------
Sample 1
coefficient of determination: 0.9493875116415998
intercept: 1.2428808726920764
slope: [0.97444493]
---------------
---------------
Sample 2
coefficient of determination: 0.9387340067647773
intercept: -0.9816686305704536
slope: [0.96570607]
---------------
---------------
Sample 3
coefficient of determination: 0.9377122883052714
intercept: -0.39283402696745284
slope: [0.94851758]
---------------
---------------
Sample 4
coefficient of determination: 0.9423996769857416
intercept: -0.9594932632116908
slope: [0.96029649]
---------------
---------------
Sample 5
coefficient of determination: 0.9413392491730915
intercept: 0.9628209481707088
slope: [0.9670781]
---------------
---------------
Sample 6
coefficient of determination: 0.9511292840954059
intercept: -0.7362373577383963
slope: [0.96670598]
---------------
---------------
Sample 7
coefficient of determination: 0.9502337758504953
intercept: 1.1111322545268152
slope: [0.95937104]
---------------
---------------
